In [1]:
%matplotlib inline

In [ ]:
import os
import sys

In [ ]:
lmu_path = os.path.abspath("../lmu")
sys.path.append(lmu_path)


In [ ]:
from lmu import LMUCell

import keras as K
from keras.applications import ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Input, GlobalAveragePooling2D, Conv2DTranspose, Reshape, Lambda, Conv2D, UpSampling2D
from keras.layers import TimeDistributed
from keras.layers.recurrent import RNN
from keras.models import Sequential, Model
from keras.initializers import Constant
from keras.utils import multi_gpu_model, to_categorical

import numpy as np

Using TensorFlow backend.


In [ ]:
def lmu_layer(return_sequences=False,**kwargs):
    return RNN(LMUCell(units=6,
                       order=6,
                       theta=15,
                       input_encoders_initializer=Constant(1),
                       hidden_encoders_initializer=Constant(0),
                       memory_encoders_initializer=Constant(0),
                       input_kernel_initializer=Constant(0),
                       hidden_kernel_initializer=Constant(0),
                       memory_kernel_initializer='glorot_normal',
                      ),
               return_sequences=return_sequences,
               **kwargs)

def Lmu_stack(input_tensor, return_sequences):
    t1 = []
    t2 = []
    shape = K.backend.int_shape(input_tensor)
    input_tensor = Reshape([shape[1], -1, shape[-1]])(input_tensor)
    for i in range(shape[-1]):
        x = Lambda(lambda x: x[...,i])(input_tensor)
        x = lmu_layer(return_sequences=return_sequences)(x)
        t1.append(Dense(1)(x))
        t2.append(Dense(49)(x))
    return [Lambda(lambda x: K.backend.stack(x, axis=-1))(t1), Lambda(lambda x: K.backend.stack(x, axis=-1))(t2)]

def velocity_layer(t):
    t = TimeDistributed(Reshape([-1]))(t)
    return TimeDistributed(Dense(2))(t)

def deconvolution_layer(t):
    heat = TimeDistributed(Reshape([7,7,-1]))(t)
    heat = TimeDistributed(Conv2DTranspose(filters=32,kernel_size=3,strides=2, dilation_rate=1,activation='relu', padding='same'))(heat)
    heat = TimeDistributed(UpSampling2D( interpolation='bilinear'))(heat)
    heat = TimeDistributed(Conv2DTranspose(filters=8,kernel_size=3,strides=2, dilation_rate=1,activation='relu', padding='same'))(heat)
    heat = TimeDistributed(UpSampling2D( interpolation='bilinear'))(heat)
    heat = TimeDistributed(Conv2DTranspose(filters=1,kernel_size=3,strides=2, dilation_rate=1,activation='relu', padding='same'))(heat)
    return heat

In [ ]:
seq_len = 15

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in resnet_model.layers:
    layer.trainable=False
    
# base_out = resnet_model.output
# base_out = Conv2D(kernel_size=1, filters=64)(base_out)
# base_model = Model(inputs=resnet_model.input, outputs=base_out)

input_layer = Input(shape=(seq_len, 224, 224, 3))

# x = TimeDistributed(base_model)(input_layer)
x = TimeDistributed(resnet_model)(input_layer)
x = TimeDistributed(Conv2D(kernel_size=1, filters=64))(x) #reduce channels

# i = Input(shape=(seq_len, 7, 7, 64))

# x = Reshape(target_shape=(seq_len, 2048))(x)
out1, out2 = Lmu_stack(x, return_sequences=True)

# v = Reshape([-1])(out1)
# v = Dense(2)(v)


# #Deconvolution, this is huge, how to fix?
# heat = Reshape([7,7,-1])(out2)
# print(heat.shape)
# heat = Conv2DTranspose(filters=32,kernel_size=3,strides=2, dilation_rate=1,activation='relu', padding='same')(heat)
# heat = UpSampling2D( interpolation='bilinear')(heat)
# heat = Conv2DTranspose(filters=8,kernel_size=3,strides=2, dilation_rate=1,activation='relu', padding='same')(heat)
# heat = UpSampling2D( interpolation='bilinear')(heat)
# heat = Conv2DTranspose(filters=1,kernel_size=3,strides=2, dilation_rate=1,activation='relu', padding='same')(heat)

v = velocity_layer(out1)

heat = deconvolution_layer(out2)

model = Model(inputs=input_layer, outputs=[heat,v])

C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
(?, 15, 1, 64)
(?, 15, 49, 64)

In [ ]:
model.summary()